DISCLAIMER: Isto não é, sob hipótese alguma, uma recomendação de investimento.
A serventia destes algorítmos é apenas para didática.

Desenvolvido em:
Python 3.8.10 64-bit
Desenvolvido por:
Fernando Luigi Cunha Dipp
Contato:
fernando.dipp@gmail.com

In [1]:
# Bibliotecas necessárias
import requests as requests
from bs4 import BeautifulSoup

In [2]:
# Site que servirá como fonte para a obtenção dos dados
Site = 'https://opcoes.net.br/opcoes/pozinhos'
Pagina = requests.request('GET', Site)
# Retornando o Http Status da requisição acima
Pagina

<Response [200]>

In [3]:
# Fazendo o parse com o Beautifulsoup
Coleta = BeautifulSoup(Pagina.text, 'html.parser')
# Declarando e inicializando as variáveis da tabela de referência
TabelaBody = Coleta.find_all('table', attrs={'id': "tbl-opcoes-body"})

In [4]:
# Listando as 20 ações com mais liquidções nos últimos 2 meses (FONTE: https://fundamentus.com.br/resultado.php)
maisNegociadas= ('VALE','PETR','ELET','ITUB','BBDC','MGLU','PETR','BBAS','PRIO','RENT','B3SA','ABEV','GGBR','HAPV','LREN','SUZB','WEGE','JBSS','BPAC','AMER')

In [5]:
# Inicializando uma lista que irá salvar as ações que passaram pelo filtro salvas tipo dicionário (lista de dicionários)
opcoesList = []

# Varredura dos dados da tabela iterando todos os itens dela
for i in range(len(TabelaBody[0].find_all('tr'))):
    # Inicializando as variáveis de cada linha
    opcao = TabelaBody[0].find_all('tr')[i].find_all('td')[0].text.strip()
    tipo = TabelaBody[0].find_all('tr')[i].find_all('td')[1].text.strip()
    vencimento = TabelaBody[0].find_all('tr')[i].find_all('td')[2].text.strip()
    strike = TabelaBody[0].find_all('tr')[i].find_all('td')[5].text.strip()
    distStrikePercent = TabelaBody[0].find_all('tr')[i].find_all('td')[6].text.strip()
    distStrikePercent = distStrikePercent.replace(',','.')
    distStrikePercent = float(distStrikePercent)
    ultimo = TabelaBody[0].find_all('tr')[i].find_all('td')[10].text.strip()
    numNegocios = TabelaBody[0].find_all('tr')[i].find_all('td')[15].text.strip()
    volFinanceiro = TabelaBody[0].find_all('tr')[i].find_all('td')[16].text.strip()
    # Iterando cada item da lista de ações mais negociadas
    for j in range(len(maisNegociadas)):
        # Comparando com o nome da empresa dona da opção
        if maisNegociadas[j] in opcao:
            # Adicionando a condição de filtro. No caso "> -10.0" está listando apenas as opções que estão entre 0% e -10% do seu respectivo strike
            if distStrikePercent > -10.0:
                # Caso achado, imprime na tela
                print(opcao + ' ' + str(distStrikePercent) + '%'+' do strike, negociado a: R$' + ultimo + ' vencendo em: ' + vencimento)
                # Inicializa um novo dicionário
                opcoesDict = {
                'Ticker': opcao,
                'Tipo' : tipo,
                'Preco' : ultimo,
                'Vencimento' : vencimento,
                'Strike' : strike,
                'Dist para Strike %' : distStrikePercent,
                'Numero de Negocios' : numNegocios,
                'Volume Financeiro' : volFinanceiro
                }
                # Adiciona o novo dicionário à lista declarada no início deste bloco
                opcoesList.append(opcoesDict)

ABEVT135 -8.5% do strike, negociado a: R$0,03 vencendo em: 19/08/2022
ABEVT137 -6.8% do strike, negociado a: R$0,03 vencendo em: 19/08/2022
BBDCT145 -6.8% do strike, negociado a: R$0,04 vencendo em: 19/08/2022
BBDCT189 -8.4% do strike, negociado a: R$0,03 vencendo em: 19/08/2022
ITUBT234 -8.6% do strike, negociado a: R$0,05 vencendo em: 19/08/2022


In [6]:
# Imprimindo os dicionários da lista
for item in range(len(opcoesList)):
    print(opcoesList[item])

{'Ticker': 'ABEVT135', 'Tipo': 'PUT', 'Preco': '0,03', 'Vencimento': '19/08/2022', 'Strike': '13,54', 'Dist para Strike %': -8.5, 'Numero de Negocios': '16', 'Volume Financeiro': '947,00'}
{'Ticker': 'ABEVT137', 'Tipo': 'PUT', 'Preco': '0,03', 'Vencimento': '19/08/2022', 'Strike': '13,79', 'Dist para Strike %': -6.8, 'Numero de Negocios': '14', 'Volume Financeiro': '2.722,00'}
{'Ticker': 'BBDCT145', 'Tipo': 'PUT', 'Preco': '0,04', 'Vencimento': '19/08/2022', 'Strike': '14,32', 'Dist para Strike %': -6.8, 'Numero de Negocios': '2', 'Volume Financeiro': '134,00'}
{'Ticker': 'BBDCT189', 'Tipo': 'PUT', 'Preco': '0,03', 'Vencimento': '19/08/2022', 'Strike': '17,00', 'Dist para Strike %': -8.4, 'Numero de Negocios': '4.858', 'Volume Financeiro': '52.057,88'}
{'Ticker': 'ITUBT234', 'Tipo': 'PUT', 'Preco': '0,05', 'Vencimento': '19/08/2022', 'Strike': '23,07', 'Dist para Strike %': -8.6, 'Numero de Negocios': '429', 'Volume Financeiro': '23.560,00'}


In [7]:
import math
disponivelInvest = 50
rentabAlvo = 100 #%
corretagem = 0
issCorretagem = 0 #%

for item in range(len(opcoesList)):
    precoOpcao = float(opcoesList[item]['Preco'].replace(',','.'))
    qntdMaxCompra = disponivelInvest / precoOpcao
    qntdMaxCompra = math.trunc(qntdMaxCompra / 100) * 100
    custoTotal = precoOpcao * qntdMaxCompra + (corretagem * (1+(issCorretagem/100)))
    breakEven = round((custoTotal + (corretagem * (1 + (issCorretagem / 100))))/qntdMaxCompra + 0.01, 2)
    stopGain = round((custoTotal + (corretagem * (1 + (issCorretagem / 100)))+((rentabAlvo / 100)*(precoOpcao*qntdMaxCompra)))/qntdMaxCompra, 2)
    lucroLiquido = round(stopGain*qntdMaxCompra-(custoTotal+(corretagem * (1+(issCorretagem/100)))),2)
    print('Possível comprar '+str(qntdMaxCompra)+' opções de '+ opcoesList[item]['Ticker'] + ', com um Break-Even de R$' "%.2f" %breakEven +
    ', lucro alvo de R$'"%.2f" %stopGain+', tendo um lucro total de R$'"%.2f" %lucroLiquido)

Possível comprar 1600 opções de ABEVT135, com um Break-Even de R$0.04, lucro alvo de R$0.06, tendo um lucro total de R$48.00
Possível comprar 1600 opções de ABEVT137, com um Break-Even de R$0.04, lucro alvo de R$0.06, tendo um lucro total de R$48.00
Possível comprar 1200 opções de BBDCT145, com um Break-Even de R$0.05, lucro alvo de R$0.08, tendo um lucro total de R$48.00
Possível comprar 1600 opções de BBDCT189, com um Break-Even de R$0.04, lucro alvo de R$0.06, tendo um lucro total de R$48.00
Possível comprar 1000 opções de ITUBT234, com um Break-Even de R$0.06, lucro alvo de R$0.10, tendo um lucro total de R$50.00


In [11]:
BSList = []
for idx in range(len(opcoesList)):
    urlBlackScholes = 'https://opcoes.net.br/calculadora-Black-Scholes/' + opcoesList[idx]['Ticker']
    blackScholes = requests.get(urlBlackScholes)
    blackScholes = BeautifulSoup(blackScholes.text, 'html.parser')
    blackScholes = blackScholes.find_all('form', attrs={'id': "frmCalc"})
    blackScholes = blackScholes[0].find_all('input')
    ticker = blackScholes[1]['value']
    strike = blackScholes[5]['value']
    txJuros = blackScholes[7]['value']
    dataRef = blackScholes[9]['value']
    precoAcao = blackScholes[13]['value']
    volImplicita = blackScholes[15]['value']
    premioOpcao = blackScholes[17]['value']
    BSDict = {
        'Ticker' : ticker,
        'Strike' : strike,
        'Taxa de Juros' : txJuros,
        'Data de Ref' : dataRef,
        'Preco da Acao' : precoAcao,
        'Volatilidade Impl' : volImplicita,
        'Premio Opcao' : premioOpcao
        }
    BSList.append(BSDict)

for item in range(len(BSList)):
    print(BSList[item])

{'Ticker': 'ABEVT135', 'Strike': '13,54', 'Taxa de Juros': '13,75', 'Data de Ref': '08/08/2022', 'Preco da Acao': '14,80', 'Volatilidade Impl': '34,25', 'Premio Opcao': '0,03'}
{'Ticker': 'ABEVT137', 'Strike': '13,79', 'Taxa de Juros': '13,75', 'Data de Ref': '08/08/2022', 'Preco da Acao': '14,80', 'Volatilidade Impl': '28,63', 'Premio Opcao': '0,03'}
{'Ticker': 'BBDCT145', 'Strike': '14,32', 'Taxa de Juros': '13,75', 'Data de Ref': '08/08/2022', 'Preco da Acao': '15,36', 'Volatilidade Impl': '29,41', 'Premio Opcao': '0,04'}
{'Ticker': 'BBDCT189', 'Strike': '17,00', 'Taxa de Juros': '13,75', 'Data de Ref': '08/08/2022', 'Preco da Acao': '18,56', 'Volatilidade Impl': '32,22', 'Premio Opcao': '0,03'}
{'Ticker': 'ITUBT234', 'Strike': '23,07', 'Taxa de Juros': '13,75', 'Data de Ref': '08/08/2022', 'Preco da Acao': '25,24', 'Volatilidade Impl': '34,34', 'Premio Opcao': '0,05'}
